The purpose of this notebook is to test the accuracy of the TF prediction 

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files

### Load Feature Data
Upload essential and non-essential gene features, or use HomoSapiens data as default (must be uploaded in directory).


##download dataset

In [ ]:
#download datset to aws server
!rm -r vulgaris
!rm vulgaris.zip
!wget 3.133.132.243/vulgaris.zip
!unzip vulgaris.zip

rm: cannot remove 'vulgaris': No such file or directory
rm: cannot remove 'vulgaris.zip': No such file or directory
--2021-04-04 22:03:36--  http://3.133.132.243/vulgaris.zip
Connecting to 3.133.132.243:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128982537 (123M) [application/zip]
Saving to: ‘vulgaris.zip’

vulgaris.zip        100%[===================>] 123.01M  4.84MB/s    in 26s     

2021-04-04 22:04:03 (4.66 MB/s) - ‘vulgaris.zip’ saved [128982537/128982537]

Archive:  vulgaris.zip
  inflating: vulgaris/EssentialFeature_collection_APAAC.csv  
  inflating: vulgaris/EssentialFeature_collection_CTDD.csv  
  inflating: vulgaris/EssentialFeature_collection_Ctriad.csv  
  inflating: vulgaris/EssentialFeature_collection_Dipetite.csv  
  inflating: vulgaris/EssentialFeature_collection_Moran.csv  
  inflating: vulgaris/EssentialFeature_collection_Moreau.csv  
  inflating: vulgaris/EssentialFeature_collection_Nucleotide.csv  
  inflating: vulgaris/EssentialFeature

### Kegg Data

In [ ]:
# Kegg data from G20 Paper
feature_data = pd.read_csv("/content/EssentialFeature_collection_Nucleotide.csv")
genes = feature_data.index
feature_data

,kmer_2_AA,kmer_2_AC,kmer_2_AG,kmer_2_AT,kmer_2_CA,kmer_2_CC,kmer_2_CG,kmer_2_CT,kmer_2_GA,kmer_2_GC,...,CBI,Fop,Nc,GC3s,GC,L_sym,L_aa,Gravy,Aromo,X
DMR_00010,66,68,48,45,73,165,144,73,77,140,...,0.170,0.514,36.14,0.802,0.646,434,446,-0.281166,0.094170,NaN
DMR_00020,52,63,68,48,81,126,114,67,82,123,...,0.237,0.558,35.13,0.860,0.622,385,397,-0.084635,0.090680,NaN
DMR_00030,128,157,140,88,172,249,264,120,176,238,...,0.248,0.563,31.83,0.873,0.629,789,804,-0.408085,0.075871,NaN
DMR_00040,141,163,101,112,197,260,271,123,142,245,...,0.365,0.624,30.74,0.868,0.624,795,813,-0.226076,0.052891,NaN
DMR_00050,37,35,37,22,37,100,84,62,48,115,...,0.172,0.509,43.00,0.764,0.667,267,268,-0.184702,0.055970,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMR_P2_00060,97,22,23,51,18,3,8,14,34,12,...,-0.160,0.336,45.34,0.196,0.264,143,149,-0.491946,0.107383,NaN
DMR_P2_00070,95,20,33,40,28,11,9,16,35,16,...,0.045,0.468,39.53,0.240,0.331,154,166,-0.485542,0.138554,NaN
DMR_P2_00080,74,36,48,44,43,40,31,36,64,43,...,0.034,0.434,56.81,0.530,0.496,219,227,-0.567401,0.030837,NaN
DMR_P2_00090,30,10,19,29,17,11,11,12,23,14,...,-0.050,0.370,47.89,0.359,0.408,92,94,-0.495745,0.106383,NaN


In [ ]:
meta_df = pd.read_excel("/content/dma_Desulfovibrio_magneticus.xlsx", sheet_name="Sheet1",engine="openpyxl")
meta_idx = meta_df['Gene_Locus']
meta_idx = pd.Series([x.upper() for x in meta_idx.values])
meta_df = meta_df.set_index(keys=meta_idx)
meta_df

,Unnamed: 0,Org,Gene_Locus,KO_Nbr,Gene_Name,Gene_Function,E_Score,P_Score,Organism,Gene_essentaility
DMR_00010,0,dma,DMR_00010,K02313,dnaA,chromosomal replication initiator protein,0.5800,0.2130,Desulfovibrio magneticus,NE
DMR_00020,1,dma,DMR_00020,K02338,"DPO3B, dnaN",DNA polymerase III subunit beta,0.8789,0.9703,Desulfovibrio magneticus,E
DMR_00070,2,dma,DMR_00070,K13038,"coaBC, dfp",phosphopantothenoylcysteine decarboxylase / ph...,0.4153,0.3095,Desulfovibrio magneticus,NE
DMR_00110,3,dma,DMR_00110,K08679,E5.1.3.6,UDP-glucuronate 4-epimerase,0.0035,0.0200,Desulfovibrio magneticus,NE
DMR_00180,4,dma,DMR_00180,K02066,"mlaE, linK",phospholipid/cholesterol/gamma-HCH transport s...,0.0012,0.0117,Desulfovibrio magneticus,NE
...,...,...,...,...,...,...,...,...,...,...
DMR_T00470,1076,dma,DMR_t00470,K14227,tRNA-Ile,tRNA Ile,0.0000,0.2024,Desulfovibrio magneticus,E
DMR_T00480,1077,dma,DMR_t00480,K14220,tRNA-Asn,tRNA Asn,0.0000,0.2024,Desulfovibrio magneticus,E
DMR_T00490,1078,dma,DMR_t00490,K14224,tRNA-Glu,tRNA Glu,0.0000,0.2024,Desulfovibrio magneticus,E
DMR_T00500,1079,dma,DMR_t00500,K14223,tRNA-Gln,tRNA Gln,0.0000,0.2024,Desulfovibrio magneticus,E


In [ ]:
# get class labels for dataset
df_full = feature_data.merge(meta_df[['Gene_essentaility']], how='inner', left_index=True, right_index=True)
#df_full.drop('X', axis = 1, inplace=True)
df_full

,kmer_2_AA,kmer_2_AC,kmer_2_AG,kmer_2_AT,kmer_2_CA,kmer_2_CC,kmer_2_CG,kmer_2_CT,kmer_2_GA,kmer_2_GC,...,Fop,Nc,GC3s,GC,L_sym,L_aa,Gravy,Aromo,X,Gene_essentaility
DMR_00010,66,68,48,45,73,165,144,73,77,140,...,0.514,36.14,0.802,0.646,434,446,-0.281166,0.094170,NaN,NE
DMR_00020,52,63,68,48,81,126,114,67,82,123,...,0.558,35.13,0.860,0.622,385,397,-0.084635,0.090680,NaN,E
DMR_00070,42,60,44,45,75,164,133,68,62,147,...,0.527,37.54,0.837,0.672,387,406,0.106897,0.076355,NaN,NE
DMR_00110,58,62,41,57,87,96,87,54,49,84,...,0.583,38.72,0.787,0.567,324,335,-0.101791,0.125373,NaN,NE
DMR_00180,21,19,22,40,44,84,72,55,27,91,...,0.494,35.13,0.827,0.639,243,256,0.855078,0.093750,NaN,NE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMR_P1_00300,36,57,44,40,63,100,118,61,64,116,...,0.593,40.02,0.806,0.652,324,337,-0.073591,0.059347,NaN,NE
DMR_P1_00310,41,58,36,39,67,141,121,58,50,119,...,0.557,36.72,0.847,0.675,334,349,-0.013181,0.074499,NaN,NE
DMR_P1_00330,60,83,51,52,103,171,120,70,65,119,...,0.585,31.93,0.848,0.629,402,417,0.029496,0.076739,NaN,NE
DMR_P1_00350,18,32,22,20,30,109,89,48,31,95,...,0.582,34.13,0.803,0.697,239,243,0.255555,0.061728,NaN,E


In [ ]:
# class mappings, 1 = Essential and 0 = Non-Essential
#mappings = {'Dispensable': 0, 'ExpectedEssential': 1, 'EdgeInsertionOnly': 0, 'Desulfovibrio-specific essential': 1, 'NotUnique': 0, 'OtherNoInsertion': 2}
mappings = {'NE': 0, 'E': 1}
classes = df_full.pop('Gene_essentaility')
essential_labels = classes.map(mappings)
df_full['essential'] = essential_labels
# remove unknowns
df_full = df_full[df_full['essential'] < 2]
df_full

,kmer_2_AA,kmer_2_AC,kmer_2_AG,kmer_2_AT,kmer_2_CA,kmer_2_CC,kmer_2_CG,kmer_2_CT,kmer_2_GA,kmer_2_GC,...,Fop,Nc,GC3s,GC,L_sym,L_aa,Gravy,Aromo,X,essential
DMR_00010,66,68,48,45,73,165,144,73,77,140,...,0.514,36.14,0.802,0.646,434,446,-0.281166,0.094170,NaN,0
DMR_00020,52,63,68,48,81,126,114,67,82,123,...,0.558,35.13,0.860,0.622,385,397,-0.084635,0.090680,NaN,1
DMR_00070,42,60,44,45,75,164,133,68,62,147,...,0.527,37.54,0.837,0.672,387,406,0.106897,0.076355,NaN,0
DMR_00110,58,62,41,57,87,96,87,54,49,84,...,0.583,38.72,0.787,0.567,324,335,-0.101791,0.125373,NaN,0
DMR_00180,21,19,22,40,44,84,72,55,27,91,...,0.494,35.13,0.827,0.639,243,256,0.855078,0.093750,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMR_P1_00300,36,57,44,40,63,100,118,61,64,116,...,0.593,40.02,0.806,0.652,324,337,-0.073591,0.059347,NaN,0
DMR_P1_00310,41,58,36,39,67,141,121,58,50,119,...,0.557,36.72,0.847,0.675,334,349,-0.013181,0.074499,NaN,0
DMR_P1_00330,60,83,51,52,103,171,120,70,65,119,...,0.585,31.93,0.848,0.629,402,417,0.029496,0.076739,NaN,0
DMR_P1_00350,18,32,22,20,30,109,89,48,31,95,...,0.582,34.13,0.803,0.697,239,243,0.255555,0.061728,NaN,1


In [ ]:
df_essential = df_full[df_full['essential'] == 1]
df_nonEssential = df_full[df_full['essential'] == 0]

In [ ]:
df_full.to_csv("PaperG20_filtered_combined-protein.csv")

### Balance classes
Due to the imbalance in essential and non-essential classes, we will oversample the essential genes and undersample the non-essential genes to achieve a 1:2 ratio between the two.

In [ ]:
df_essential_oversample = pd.concat([df_essential, df_essential], ignore_index=True)
# sample non-essential genes
total_essential_samples = len(df_essential_oversample)
df_nonE_sample = df_nonEssential.sample(2*total_essential_samples)

# combine essential and non-essential sets, drop gene name column
df_full = pd.concat([df_essential_oversample.iloc[:,1:],df_nonE_sample.iloc[:,1:]], ignore_index=True)

# display full feature data
df_full

,kmer_2_AC,kmer_2_AG,kmer_2_AT,kmer_2_CA,kmer_2_CC,kmer_2_CG,kmer_2_CT,kmer_2_GA,kmer_2_GC,kmer_2_GG,...,Fop,Nc,GC3s,GC,L_sym,L_aa,Gravy,Aromo,X,essential
0,63,68,48,81,126,114,67,82,123,105,...,0.558,35.13,0.860,0.622,385,397,-0.084635,0.090680,NaN,1
1,61,50,44,66,139,148,82,68,164,115,...,0.571,33.37,0.837,0.653,410,421,-0.122803,0.073634,NaN,1
2,85,98,66,114,216,194,102,115,205,143,...,0.572,30.39,0.884,0.650,568,592,-0.223649,0.092905,NaN,1
3,161,127,95,170,296,300,150,177,308,246,...,0.578,30.16,0.902,0.659,843,888,-0.192342,0.092342,NaN,1
4,34,35,28,41,84,115,47,45,116,106,...,0.507,38.54,0.832,0.690,280,289,0.156401,0.076125,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,44,28,21,45,101,110,49,44,113,115,...,0.516,38.80,0.859,0.715,277,283,-0.121202,0.045936,NaN,0
692,110,99,50,118,302,308,96,127,312,247,...,0.534,34.02,0.878,0.719,719,734,-0.323978,0.061308,NaN,0
693,78,77,49,79,129,139,70,107,127,90,...,0.683,25.88,0.879,0.624,404,426,-0.319953,0.082160,NaN,0
694,26,22,17,27,86,63,47,32,76,62,...,0.515,34.03,0.874,0.690,198,204,0.132843,0.034314,NaN,0


## Build training and validation data

In [ ]:
import tensorflow as tf

# create a training and testing subset of the input data
df_test = df_full.sample(50)
df_train = df_full.drop(df_test.index)
test_target = df_test.pop('essential')
train_target = df_train.pop('essential')

# batch and shuffle a Dataset for Deep Learning Model
target = df_full.pop("essential")
dataset = tf.data.Dataset.from_tensor_slices((df_full.values, target.values))
dataset = dataset.shuffle(len(df_full)).batch(1)

## Model 1: Deep Learning
Construct a Deep Learning model to predict essential genes from feature matrix.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

# create a model checkpoint for saving model with highest accuracy
filepath = "/content/vulgaris/essential_gene_predictor_model_Proteins.h5"

earlystopper = EarlyStopping(monitor='binary_accuracy',mode='max',patience=10, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='binary_accuracy',verbose=1, 
                             save_best_only=True, mode='max')

callbacks_list = [earlystopper, checkpoint]

# Use binary cross-entropy for loss function and various accuracy metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate =0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.binary_accuracy,
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

In [ ]:
model.fit(x=dataset, epochs=100, callbacks=callbacks_list)

Epoch 1/100
2322/2322 [==============================] - 16s 5ms/step - loss: 1188.9933 - binary_accuracy: 0.5542 - precision: 0.3152 - recall: 0.3288

Epoch 00001: binary_accuracy improved from -inf to 0.55512, saving model to /content/paperwriting/essential_gene_predictor_model_Proteins.h5
Epoch 2/100
2322/2322 [==============================] - 12s 5ms/step - loss: 308.5359 - binary_accuracy: 0.5542 - precision: 0.3190 - recall: 0.3268

Epoch 00002: binary_accuracy improved from 0.55512 to 0.56460, saving model to /content/paperwriting/essential_gene_predictor_model_Proteins.h5
Epoch 3/100
2322/2322 [==============================] - 12s 5ms/step - loss: 26.6448 - binary_accuracy: 0.5562 - precision: 0.3480 - recall: 0.3700

Epoch 00003: binary_accuracy improved from 0.56460 to 0.57623, saving model to /content/paperwriting/essential_gene_predictor_model_Proteins.h5
Epoch 4/100
2322/2322 [==============================] - 12s 5ms/step - loss: 0.7499 - binary_accuracy: 0.6503 - preci

### Training Results

In [ ]:
trained_model = tf.keras.models.load_model("/content/vulgaris/essential_gene_predictor_model_Proteins.h5")
trained_model.evaluate(dataset, verbose=1)

In [ ]:
trained_model.evaluate(df_test, verbose = 1)

NameError: ignored

In [ ]:
# test the model on a random sample of the non-essential data
test_nonE_sample = df_nonEssential.sample(500).iloc[:,1:]
test_target = test_nonE_sample.pop("essential")
trained_model.evaluate(x = test_nonE_sample.values, y=test_target.values)

16/16 [==============================] - 0s 5ms/step - loss: 0.4257 - binary_accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00


[0.42567864060401917, 1.0, 0.0, 0.0]

In [ ]:
# test the model on the essential gene data
test_E_sample = df_essential_oversample.iloc[:,1:-1]
E_target = df_essential_oversample["essential"]
trained_model.evaluate(x=test_E_sample.values, y=E_target.values)

25/25 [==============================] - 0s 5ms/step - loss: 1.0563 - binary_accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00


[1.0562725067138672, 0.0, 0.0, 0.0]

## Model 2: Linear Regression

### Feature Preprocessing

In [ ]:
DATA_COLUMNS = df_full.columns.values

feature_columns = []
for feature_name in DATA_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))

In [ ]:
# Define input function for training and evaluation
def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    if shuffle:
      dataset = dataset.shuffle(len(X))
    dataset = dataset.repeat(n_epochs)
    dataset = dataset.batch(len(X))
    return dataset
  return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(df_train, train_target)
eval_train_fn = make_input_fn(df_train, train_target, n_epochs=1, shuffle=False)
eval_test_fn = make_input_fn(df_test, test_target, n_epochs=1, shuffle=False)

### Train and Evaluate Linear Model

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns, optimizer=tf.keras.optimizers.Ftrl(learning_rate=0.001,l2_regularization_strength=0.001))

# Train model.
linear_est.train(train_input_fn, max_steps=1500)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptr_s8rs7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmptr_s8rs7/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: ignored

In [ ]:
# Evaluation.
result = linear_est.evaluate(eval_test_fn)
print(pd.Series(result))

NameError: ignored

In [ ]:
# Plot histogram of predictions
pred_dicts = linear_est.predict(eval_test_fn)
lin_probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

lin_probs.plot(kind='hist', bins=20, title='predicted probabilities')

NameError: ignored

### Model 3: Decision Tree Gradient Boosting

In [ ]:
n_batches = 1
est = tf.estimator.BoostedTreesClassifier(feature_columns,
                                          n_batches_per_layer=n_batches, n_trees = 100, max_depth = 8, center_bias = True, learning_rate=0.01, l2_regularization=0.001)

est.train(train_input_fn, max_steps=200)

NameError: ignored

In [ ]:
# Evaluate the model on the testing set
result = est.evaluate(eval_test_fn)
print(pd.Series(result))

NameError: ignored

In [ ]:
# create a histogram of predictions for the test set
pred_dicts = est.predict(eval_test_fn)
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

NameError: ignored

In [ ]:
# test the model on a random sample of the non-essential data
test_nonE_sample = df_nonEssential.sample(50).iloc[:,1:]
test_target = test_nonE_sample.pop("essential")
test_nonE_fn = make_input_fn(test_nonE_sample, test_target, shuffle=False, n_epochs=1)

# Evaluate the model on the testing set
result = est.evaluate(test_nonE_fn)
print(pd.Series(result))

# plot results
pred_dicts = est.predict(test_nonE_fn)
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

NameError: ignored

## Model 4: Random Forest Classifier

In [ ]:
# rebalance the classes
df_nonE_sample_RF = df_nonEssential.sample(2*total_essential_samples)

# combine essential and non-essential sets, drop gene name column
df_RF = pd.concat([df_essential_oversample.iloc[:,1:],df_nonE_sample_RF.iloc[:,1:]], ignore_index=True)

# create a training and testing subset of the input data
df_test_RF = df_RF.sample(100)
df_train_RF = df_RF.drop(df_test_RF.index)
test_target_RF = df_test_RF.pop('essential')
train_target_RF = df_train_RF.pop('essential')

Rescale the input data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Custom scaler function
def standardScaling(feature):
   scaler = StandardScaler().fit(feature)
   scaled_feature = scaler.transform(feature)
   scaled_feat = pd.DataFrame(data = scaled_feature, columns = df_train_RF.columns)
   return scaled_feat
# Calling the scaler function by passing training and testing sets to get the scaled data set
X_train_scaled = standardScaling(df_train_RF)
X_test_scaled = standardScaling(df_test_RF)
# Print X_train scaled data
X_train_scaled.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in 

,kmer_2_AC,kmer_2_AG,kmer_2_AT,kmer_2_CA,kmer_2_CC,kmer_2_CG,kmer_2_CT,kmer_2_GA,kmer_2_GC,kmer_2_GG,...,CBI,Fop,Nc,GC3s,GC,L_sym,L_aa,Gravy,Aromo,X
0,-0.033217,0.536065,0.161704,0.137417,-0.147347,-0.232858,0.003626,0.365762,-0.168392,-0.095884,...,0.094282,0.234604,0.066880,0.217415,-0.756790,0.026206,0.018709,-0.332808,0.929109,NaN
1,-0.081327,0.022408,0.023729,-0.174229,0.008190,0.209252,0.380532,0.044237,0.353729,0.058109,...,0.501300,0.480541,-0.323973,-0.132676,-0.052845,0.140080,0.124316,-0.452109,0.206112,NaN
2,0.495992,1.392160,0.782594,0.823040,0.929451,0.807400,0.883073,1.123642,0.875850,0.489289,...,0.501300,0.499459,-0.985759,0.582726,-0.120969,0.859761,0.876771,-0.767322,1.023481,NaN
3,2.324166,2.219718,1.782915,1.986522,1.886604,2.185742,2.089172,2.547539,2.187519,2.075416,...,0.594333,0.612968,-1.036837,0.856710,0.083402,2.112371,2.179265,-0.669467,0.999602,NaN
4,-0.730810,-0.405640,-0.528172,-0.693641,-0.649853,-0.219855,-0.498916,-0.483983,-0.257535,-0.080485,...,-0.661610,-0.730222,0.824158,-0.208782,0.787347,-0.452063,-0.456526,0.420595,0.311766,NaN


### Build and Train RF model

In [ ]:
y_train = train_target_RF
y_test = test_target_RF

rf_model = RandomForestClassifier(n_estimators=200)
rf_model.fit(X_train_scaled, y_train)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

ValueError: ignored

### Evaluate Model

In [ ]:
# Prediction using Random Forest Model
rf_prediction = rf_model.predict(X_test_scaled)

# Evaluate the model on the test data
print('Classification Report: \n')
print(classification_report(y_test,rf_prediction))

# display actual vs. predicted values
%load_ext google.colab.data_table
rf_pred_table = pd.DataFrame({'Predicted': rf_prediction, 'Actual': y_test})
rf_pred_table

ValueError: ignored

In [ ]:
# Evaluate the model on the full  dataset
df_complete = pd.concat([df_essential_oversample.iloc[:,1:],df_nonEssential.iloc[:,1:]], ignore_index=True)
target_complete = df_complete.pop("essential")
complete_pred = rf_model.predict(standardScaling(df_complete))

# Evaluate the model on the full dataset
print('Classification Report: \n')
print(classification_report(target_complete,complete_pred))

Classification Report: 

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2510
         1.0       1.00      1.00      1.00       774

    accuracy                           1.00      3284
   macro avg       1.00      1.00      1.00      3284
weighted avg       1.00      1.00      1.00      3284

